In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import keras.models
import numpy as np

wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "c:\Users\bbaki\miniconda3\envs\fhnw\Lib\site-packages\wandb\integration\keras\keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "c:\Users\bbaki\miniconda3\envs\fhnw\Lib\site-packages\wandb\integration\keras\keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "c:\Users\bbaki\miniconda3\envs\fhnw\Lib\site-packages\wandb\integration\keras\keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.

In [2]:
#Fix the random generator seeds for better reproducibility
tf.random.set_seed(67)
np.random.seed(67)

In [30]:
    #Class to build the Model
class Model_builder:
        def __init__(self, input_shape, num_classes):
            self.input_shape = input_shape
            self.num_classes = num_classes
            self.model = None
        def build(self):

        

            input_img = Input(shape=self.input_shape)
            x= input_img
            # AUTOENCODER: how stacked should it be/How many layers? 
            #1st layer
            for _ in range(5):
                x = Conv2D(
                    filters=32,
                    kernel_size=3, #could be changed manually
                    activation='relu',
                    padding='same',
                    activity_regularizer=regularizers.l1(0.0001),
                    kernel_initializer='he_normal'
                )(x)
                x = MaxPooling2D((2, 2), padding='same')(x) #max pooling layer

            #max pooling layer to give us the result of the encoding process: latent space
            encoded = Conv2D(32, (3, 3), activation='relu', padding='same')(x)

            flattened = tf.keras.layers.Flatten()(encoded)
            dense_1 = tf.keras.layers.Dense(1344, activation='relu')(flattened)
            dense_2 = tf.keras.layers.Dense(100, activation='relu')(dense_1)
            output = tf.keras.layers.Dense(2, activation='softmax')(dense_2)
                   
                        
                        
                        #start of decoding, remove comment for decoding
                        #x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
                        #x = UpSampling2D((2, 2))(x)
                        #x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
                        #x = UpSampling2D((2, 2))(x)
                        #decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)


            #PRETRAINED EFFICIENTNET, could possibly use smaller but less computationaly expensive models: try,
            # note that use of LSTM is computationally more expensive and is not sure to bring better accuracy but still worth trying 
            #also note the use of tranfer learning here, but could also go with the route of training the whole model: after HP tuning with validation data ofc
            #efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=self.input_shape)
            #for layer in efficientnet.layers[:-20]:  #freezing everything except top 20 layers, again could also go for layer.trainable=true for all layers: do we have enough computational resources?
                #layer.trainable = False
            #x = efficientnet(encoded) #!!!!!!!!!!!!!!!!!!!!!!!change to encoded, change dimensions in other places accordingly
            #x = GlobalAveragePooling2D()(x) #helps reduce overfitting by reducing the total number of parameters in the model
            #x = Dense(32,kernel_initializer='glorot_uniform')(x) #A fully connected dense layer, possibly for feature extracion, could think of adding more layers here for feature classifications in the future
            #x = Dropout(0.3)(x)#for regularization, again use of tuning: maybe too much?
            #output = Dense(self.num_classes, activation='softmax')(x)#fully connected dense layer with softmax activation for producing the output probabilities of classes
                #This works as the classifier in our ML pipeline thanks to softmax


            #create the model using keras Model function with all the earlier configuration and return it
            self.model = tf.keras.models.Model(inputs=input_img, outputs=output)

        
        #compile the model
        def compile(self):
            self.model.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy']
            )
            return self.model
            
        

In [37]:
#Create a model_builder object, build and compile the model inside it and retrieve it as "model"
model_builder_object = Model_builder(input_shape=(240, 193, 1) ,num_classes=2)#this part can be replaced with correct input shape: 240x193x1
model_builder_object.build() 
model = model_builder_object.compile()
 


In [38]:
directory = r"C:\Users\bbaki\Desktop\fhnw\Scripts\radio_sunburst_detector\data"
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="binary",
    class_names=None,
    color_mode="grayscale",
    batch_size=32,
    image_size=(240, 193),
    shuffle=True,
    seed=42, #can change
    validation_split=0.2, #can change
    subset="training",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="binary",
    class_names=None,
    color_mode="grayscale",
    batch_size=32,
    image_size=(240, 193),
    shuffle=True,
    seed=42, #can change
    validation_split=0.2, #can change
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)





Found 100 files belonging to 2 classes.


Using 80 files for training.
Found 100 files belonging to 2 classes.
Using 20 files for validation.


In [39]:
#rescale training and validation data
def rescale(image, label):
    return image/255. , label

train_ds = train_ds.map(rescale)
validation_ds = validation_ds.map(rescale)

In [40]:
#Determine the number of images in train and validation datasets
train_total_size = tf.data.experimental.cardinality(train_ds).numpy()
val_total_size = tf.data.experimental.cardinality(validation_ds).numpy()

print("Training data Image number: ", train_total_size)
print("Validation data Image number: ", val_total_size)

Training data Image number:  3
Validation data Image number:  1


In [ ]:
#Shuffle the data for the train test split to 
#dataset = dataset.shuffle(total_size)

#Split into train and test sets
#train_dataset = dataset.take(train_size)
#test_dataset = dataset.skip(train_size)

#Function to split dataset into inputs (x) and labels (y)
#def split_into_inputs_and_labels(image, label):
#    return image, label

#Apply this function to both datasets using map function: Iteratively apply the split_into_inputs_and_labels element to 
#every element in train_dataset and test_dataset which consist of image and label thanks to keras image_dataset_from_directory function
#train_x, train_y = zip(*train_dataset.map(split_into_inputs_and_labels))
#test_x, test_y = zip(*test_dataset.map(split_into_inputs_and_labels))

In [42]:
#Train the model
model.fit(
    train_ds,
    validation_data=(validation_ds),
    epochs=10,
    batch_size=32,
)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\bbaki\miniconda3\envs\fhnw\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\bbaki\miniconda3\envs\fhnw\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\bbaki\miniconda3\envs\fhnw\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\bbaki\miniconda3\envs\fhnw\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\bbaki\miniconda3\envs\fhnw\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\bbaki\miniconda3\envs\fhnw\Lib\site-packages\keras\src\engine\input_spec.py", l